In [1]:
import ee
import pandas as pd
from prophet import Prophet
import streamlit as st
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import geemap

In [2]:
ee.Initialize()
start_date = '2021-01-01'
end_date = '2021-01-31'

In [3]:

def calculateFAI(aoi, start_date, end_date):
    # Applying filtering and cloud masking to Sentinel-2 Image Collection
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    # NDVI calculation and time series data frame generation functions
    def calculate_fai(image):
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        lambda_nir = 832.8
        lambda_red = 664.6
        lambda_swir1 = 1613.7

        # Selecting vand.
        red = image.select('B4')   # Red vand
        nir = image.select('B8')   # NIR vand
        swir1 = image.select('B11') # SWIR1 vand
        # Calculate FAI
        fai = nir.subtract(red).add(
            swir1.subtract(red).multiply(
                (lambda_nir - lambda_red) / (lambda_swir1 - lambda_red)
            )
        ).rename('FAI')
        mean_fai = fai.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=10  
        ).get('FAI')
        return ee.Feature(None, {'ds': date, 'y': mean_fai})
    
    
def process_image(img):
    img = calculateFAI(img, start_date, end_date)

    # img = cloud_mask(img)  # 구름 마스킹은 현재 주석 처리되어 있습니다.
    return img

In [4]:
aoi = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
])
# 예제 이미지에 함수 적용 (이미지 ID는 적절하게 변경해야 함)
example_image_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2020-01-01', '2020-03-30')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
    .filterBounds(aoi)

# ImageCollection에서 첫 번째 이미지 선택
example_image = example_image_collection.first()  # 첫 번째 이미지를 선택



processed_image = process_image(example_image)
# 지도 생성 및 시각화
# # 처리된 이미지에 수역 마스킹 적용
# processed_image = cloud_mask(processed_image)

Map = geemap.Map()
Map.centerObject(aoi, 14)  # 신두리사구를 중심으로 설정


# 처리된 이미지를 지도에 추가 (수중식물 지수 사용) - 수상만 표시
uvi_params = {
    'bands': ['FAI'],  # UVI 밴드만 사용
    'min': -0.4, # 수중식물 지수의 최소값
    'max': 1,   # 수중식물 지수의 최대값
    # 'palette': ['purple', 'blue', 'green', 'yellow', 'red']  # 색상 팔레트 설정
    'palette': ['red','yellow','green','blue','purple']  # 색상 팔레트 설정
}
Map.addLayer(processed_image, uvi_params, 'Processed Image (FAI)')

Map

Map(center=[35.05795764611553, 128.9470480000001], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
## 건우 코드

In [15]:
import ee
import pandas as pd

ee.Initialize()

# 관심 지역 및 기간 설정
aoi = ee.Geometry.Polygon(
    [[[
        128.852806, 35.003003
    ], [
        128.852806, 35.044895
    ], [
        128.940868, 35.044895
    ], [
        128.940868, 35.003003
    ], [
        128.852806, 35.003003
    ]]]
)
start_date = '2021-01-01'
end_date = '2023-10-31'

# Sentinel-2 ImageCollection 필터링
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

# NDWI와 FAI 계산
def calculate_indices(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    lambda_nir = 832.8
    lambda_red = 664.6
    lambda_swir1 = 1613.7

    red = image.select('B4')   # Red band
    nir = image.select('B8')   # NIR band
    swir1 = image.select('B11') # SWIR1 band

    # Calculate FAI
    fai = nir.subtract(red).add(
        swir1.subtract(red).multiply(
            (lambda_nir - lambda_red) / (lambda_swir1 - lambda_red)
        )
    ).rename('FAI')
    return image.addBands([ndwi, fai])

# NDWI가 0.5 이상이고 FAI가 50 이상인 지역의 면적 계산
def calculate_area(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mask = image.select('NDWI').gt(0.1).And(image.select('FAI').gt(-50))
    area = mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=10  # Sentinel-2 픽셀 해상도
    ).get('NDWI')
    return ee.Feature(None, {'date': date, 'area': area})

# 이미지 컬렉션에 NDWI와 FAI 계산 적용
processed_collection = sentinel2.map(calculate_indices)
area_series = processed_collection.map(calculate_area)



# 평균 NDWI 계산
mean_ndwi = processed_collection.mean().select('NDWI')


# 결과를 서버 측 객체로 변환 (Python 클라이언트로 가져오기)
area_features = area_series.getInfo()['features']

# 결과를 pandas DataFrame으로 변환
df = pd.DataFrame([feat['properties'] for feat in area_features])

# DataFrame을 'date' 컬럼에 따라 오름차순으로 정렬
df = df.sort_values(by='date')












In [16]:
import geemap

# Map 객체 생성
Map = geemap.Map()

# NDWI 시각화 매개변수 설정
ndwi_params = {'min': 0.1, 'max': 1.0, 'palette': ['blue', 'green']}
# NDWI 레이어 추가
Map.addLayer(mean_ndwi, ndwi_params, 'Mean NDWI')

# 관심 지역에 지도 중심 설정
Map.centerObject(aoi)

# 지도 표시
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
## 구름 제거 정합 이미지 생성
import ee
import pandas as pd
from prophet import Prophet
import streamlit as st
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import folium
import geemap

In [11]:
ee.Initialize()
aoi_1 = ee.Geometry.Polygon(
    [[
                            128.852806,
                            35.003003
                        ],
                        [
                            128.852806,
                            35.044895
                        ],
                        [
                            128.940868,
                            35.044895
                        ],
                        [
                            128.940868,
                            35.003003
                        ],
                        [
                            128.852806,
                            35.003003
                        ]]
)

In [16]:
start_date = '2021-01-01'
end_date = '2021-01-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100


def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))
    
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)




# 날짜 범위 생성
start_date = '2021-01-01'
end_date = '2021-01-31'

dates = pd.date_range(start=start_date, end=end_date, freq='MS')  # MS는 월의 시작을 의미합니다.

# 최종 이미지 컬렉션을 생성합니다.
final_image_collection = ee.ImageCollection([])

for start in dates:
    end = start + pd.offsets.MonthEnd()
    s2_sr_cld_col = get_s2_sr_cld_col(aoi_1, start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))
    s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median())
    # 최종 이미지 컬렉션에 추가
    final_image_collection = final_image_collection.merge(ee.ImageCollection(s2_sr_median))
    
def calculate_moisture(img):
    moisture = img.normalizedDifference(['B8A', 'B11'])
    return img.addBands(moisture.rename('moisture'))

def calculate_NDWI(img):
    NDWI = img.normalizedDifference(['B3', 'B8'])
    return img.addBands(NDWI.rename('NDWI'))

def water_bodies_index(img):
    moisture = img.select('moisture')
    NDWI = img.select('NDWI')
    water_bodies = NDWI.subtract(moisture).divide(NDWI.add(moisture))
    return img.addBands(water_bodies.rename('water_bodies'))

def calculate_water_plants(img):
    water_plants = img.normalizedDifference(['B5', 'B4'])
    NIR2 = img.expression(
        'B4 + (B11 - B4) * ((832.8 - 664.6) / (1613.7 - 664.6))',
        {
            'B4': img.select('B4'),
            'B11': img.select('B11')
        }
    )
    FAI = img.select('B8').subtract(NIR2)
    return img.addBands(water_plants.rename('water_plants')).addBands(FAI.rename('FAI'))

def calculate_wevi(img):
    # WEVI 계산: (B3 - B4) / (B3 + B4 - B2)
    green = img.select('B3')
    red = img.select('B4')
    blue = img.select('B2')
    wevi = green.subtract(red).divide(green.add(red).subtract(blue))
    return img.addBands(wevi.rename('WEVI'))

def calculate_wavi(img, alpha=1, beta=0.1):
    # WAVI 계산: (B8 - B4) / (B8 + B4 - α * B2 + β)
    NIR = img.select('B8')  # Near-Infrared
    RED = img.select('B4')  # Red
    BLUE = img.select('B2')  # Blue
    wavi = NIR.subtract(RED).divide(NIR.add(RED).subtract(BLUE.multiply(alpha)).add(beta)).rename('WAVI')
    return img.addBands(wavi)

def calculate_ndvi(img):
    # NDVI 계산: (B8 - B4) / (B8 + B4)
    NIR = img.select('B8')  # Near-Infrared
    RED = img.select('B4')  # Red
    ndvi = NIR.subtract(RED).divide(NIR.add(RED)).rename('NDVI')
    return img.addBands(ndvi)

def diff_bg(img):
    # NDVI 계산: (B8 - B4) / (B8 + B4)
    Green = img.select('B3')  # Near-Infrared
    Blue = img.select('B4')  # Red
    diff_bg = Green.subtract(Blue).rename('DIFF_BG')
    return img.addBands(diff_bg)

def mask_water_bodies(img):
    # 'water_bodies' 밴드 선택
    # water_bodies = img.select('water_bodies')
    ndvi = img.select('NDVI')

    # 수역을 나타내는 마스크 생성 (예: 'water_bodies' 값이 0보다 큰 픽셀)
    water_mask = ndvi.lt(0.8)
    # 마스크를 적용하여 수역만을 포함하는 이미지 반환
    return img.updateMask(water_mask)

def process_image(img):
    img = calculate_moisture(img)
    img = calculate_NDWI(img)
    img = water_bodies_index(img)
    img = calculate_wavi(img)
    img = calculate_wevi(img)
    img = calculate_ndvi(img)
    img = calculate_water_plants(img)
    img = diff_bg(img)
    # img = cloud_mask(img)  # 구름 마스킹은 현재 주석 처리되어 있습니다.
    return img
# ImageCollection에서 첫 번째 이미지 선택
example_image = final_image_collection.first()  # 첫 번째 이미지를 선택

processed_image = process_image(example_image)

Map = geemap.Map()
Map.centerObject(aoi_1)  # 신두리사구를 중심으로 설정

# 육지를 마스킹하기 위한 조건 설정
# water_mask = processed_image.select('UVI').gt(0.5)  # 예시로 0.5보다 큰 값인 픽셀을 수상으로 간주

# 처리된 이미지를 지도에 추가 (수중식물 지수 사용) - 수상만 표시
uvi_params = {
    'bands': ['NDWI'],  # UVI 밴드만 사용
    'min': -0.4, # 수중식물 지수의 최소값
    'max': 1,   # 수중식물 지수의 최대값
    # 'palette': ['purple', 'blue', 'green', 'yellow', 'red']  # 색상 팔레트 설정
    'palette': ['#ffffb2','#fecc5c','#fd8d3c','#f03b20','#bd0026']  # 색상 팔레트 설정
}

def mask_for_aoi(img, aoi):
    # AOI에 대한 마스크를 생성합니다.
    aoi_mask = ee.Image.constant(1).clip(aoi)
    # 이 마스크를 이미지에 적용합니다.
    return img.updateMask(aoi_mask)

# # ImageCollection에서 첫 번째 이미지 선택
# example_image = final_image_collection.first()  # 첫 번째 이미지를 선택
# processed_image = process_image(example_image)

# # AOI에 대한 마스크 적용
# masked_image_aoi1 = mask_for_aoi(processed_image, aoi_1)

Map = geemap.Map()
Map.centerObject(aoi_1)  # 지도를 AOI로 중심을 맞춥니다.

# 처리된 이미지를 지도에 추가합니다 (AOI 마스크 적용)
Map.addLayer(masked_image_aoi1, uvi_params, 'Processed Image for AOI 1 (FAI)')

# 지도를 화면에 표시
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…